# Metrics Evaluation

Python notebook for evaluation of metrics for ACL submission

## ⬇️ Imports

In [1]:
%pip install irrCAC krippendorff


[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split, KFold
import scipy.optimize as opt
from itertools import combinations
from tqdm import tqdm

SEED = 333

## 📥 Load Data

In [2]:
df = pd.read_json('/home/mlynatom/data/claim-extraction-acl24/fluency/fluency_final_results.jsonl', lines=True)
#df = pd.read_json('/home/mlynatom/data/claim-extraction-acl24/fluency/fluency_final_results_norm_levenshtein.jsonl', lines=True)
#df = pd.read_json('/home/mlynatom/data/claim-extraction-acl24/fluency/fluency_final_results_perplexity_compare.jsonl', lines=True)
df

,sentence_id,model,generated,F,F_lg,F_bert,D,A,A_luke,annotation,...,pplxt_generated_coedit_xl,identity_l,identity_xl,bpeslor,scribendi_score_l,scribendi_score_xl,jaccard_bigram_l,jaccard_bigram_xl,levenshtein_l,levenshtein_xl
0,6140,t5_small_diverse_7_beam_search,R. Kelly is a music artist.,0.997425,0.999607,0.897497,True,True,True,NaN,...,79.406815,False,False,2.267950,1,1,0.500000,0.571429,6,8
1,6140,t5_small_diverse_7_beam_search,R. Kelly has been a guest vocalist for Nas.,0.997547,0.997142,0.872724,False,True,True,NaN,...,63.075047,False,True,3.357871,-1,0,0.666667,1.000000,6,0
2,6140,t5_small_diverse_7_beam_search,R. Kelly has released collaboration albums wit...,0.996436,0.998380,0.874548,True,False,True,NaN,...,84.100304,False,False,3.399311,-1,-1,0.625000,0.625000,14,14
3,6140,t5_small_diverse_7_beam_search,R. Kelly released collaboration albums with Ja...,0.996924,0.997682,0.875569,True,False,True,NaN,...,39.236511,False,False,3.170562,1,-1,0.333333,0.076923,12,29
4,6140,t5_small_diverse_7_beam_search,R. Kelly is one of the best-selling music arti...,0.994441,0.994815,0.879582,True,True,True,NaN,...,16.860781,False,False,3.832293,-1,-1,0.133333,0.117647,66,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15555,3541,t5_small_multiclaim,Adrianne Palicki was born in 1983.,0.997649,0.999215,0.892769,True,True,True,3311.0,...,41.039127,False,False,5.651093,-1,-1,0.500000,0.500000,15,15
15556,3541,t5_small_multiclaim,Adrianne Palicki is an American.,0.999051,0.999498,0.902723,True,True,True,3311.0,...,18.124359,False,False,5.085773,-1,1,0.176471,0.500000,64,6
15557,3541,t5_small_multiclaim,Adrianne Palicki is best known for her role as...,0.998119,0.995047,0.896891,False,True,True,3311.0,...,50.003304,False,False,5.651744,-1,-1,0.727273,0.500000,15,20
15558,3541,t5_small_multiclaim,Adrianne Palicki is best known for her role in...,0.165706,0.313146,0.898684,False,True,True,1311.0,...,47.368946,False,False,5.373777,-1,-1,0.700000,0.454545,15,20


In [3]:
df2 = df
df2

,sentence_id,model,generated,F,F_lg,F_bert,D,A,A_luke,annotation,...,pplxt_generated_coedit_xl,identity_l,identity_xl,bpeslor,scribendi_score_l,scribendi_score_xl,jaccard_bigram_l,jaccard_bigram_xl,levenshtein_l,levenshtein_xl
0,6140,t5_small_diverse_7_beam_search,R. Kelly is a music artist.,0.997425,0.999607,0.897497,True,True,True,NaN,...,79.406815,False,False,2.267950,1,1,0.500000,0.571429,6,8
1,6140,t5_small_diverse_7_beam_search,R. Kelly has been a guest vocalist for Nas.,0.997547,0.997142,0.872724,False,True,True,NaN,...,63.075047,False,True,3.357871,-1,0,0.666667,1.000000,6,0
2,6140,t5_small_diverse_7_beam_search,R. Kelly has released collaboration albums wit...,0.996436,0.998380,0.874548,True,False,True,NaN,...,84.100304,False,False,3.399311,-1,-1,0.625000,0.625000,14,14
3,6140,t5_small_diverse_7_beam_search,R. Kelly released collaboration albums with Ja...,0.996924,0.997682,0.875569,True,False,True,NaN,...,39.236511,False,False,3.170562,1,-1,0.333333,0.076923,12,29
4,6140,t5_small_diverse_7_beam_search,R. Kelly is one of the best-selling music arti...,0.994441,0.994815,0.879582,True,True,True,NaN,...,16.860781,False,False,3.832293,-1,-1,0.133333,0.117647,66,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15555,3541,t5_small_multiclaim,Adrianne Palicki was born in 1983.,0.997649,0.999215,0.892769,True,True,True,3311.0,...,41.039127,False,False,5.651093,-1,-1,0.500000,0.500000,15,15
15556,3541,t5_small_multiclaim,Adrianne Palicki is an American.,0.999051,0.999498,0.902723,True,True,True,3311.0,...,18.124359,False,False,5.085773,-1,1,0.176471,0.500000,64,6
15557,3541,t5_small_multiclaim,Adrianne Palicki is best known for her role as...,0.998119,0.995047,0.896891,False,True,True,3311.0,...,50.003304,False,False,5.651744,-1,-1,0.727273,0.500000,15,20
15558,3541,t5_small_multiclaim,Adrianne Palicki is best known for her role in...,0.165706,0.313146,0.898684,False,True,True,1311.0,...,47.368946,False,False,5.373777,-1,-1,0.700000,0.454545,15,20


In [4]:
df2.columns

Index(['sentence_id', 'model', 'generated', 'F', 'F_lg', 'F_bert', 'D', 'A',
       'A_luke', 'annotation', 'F_gold', 'G_gold', 'D_gold', 'A_gold',
       'generated_coedit_l', 'generated_coedit_xl', 'pplxt_generated',
       'pplxt_generated_coedit_l', 'pplxt_generated_coedit_xl', 'identity_l',
       'identity_xl', 'bpeslor', 'scribendi_score_l', 'scribendi_score_xl',
       'jaccard_bigram_l', 'jaccard_bigram_xl', 'levenshtein_l',
       'levenshtein_xl'],
      dtype='object')

In [5]:
df2.drop(columns=["identity_l", "identity_xl", "levenshtein_l", "levenshtein_xl", "jaccard_bigram_l", "jaccard_bigram_xl", 'generated_coedit_l', 'generated_coedit_xl', 'pplxt_generated_coedit_l', 'pplxt_generated_coedit_xl', 'bpeslor', 'scribendi_score_xl'], inplace=True)

In [6]:
df2

,sentence_id,model,generated,F,F_lg,F_bert,D,A,A_luke,annotation,F_gold,G_gold,D_gold,A_gold,pplxt_generated,scribendi_score_l
0,6140,t5_small_diverse_7_beam_search,R. Kelly is a music artist.,0.997425,0.999607,0.897497,True,True,True,NaN,NaN,NaN,NaN,NaN,88.827202,1
1,6140,t5_small_diverse_7_beam_search,R. Kelly has been a guest vocalist for Nas.,0.997547,0.997142,0.872724,False,True,True,NaN,NaN,NaN,NaN,NaN,63.083439,-1
2,6140,t5_small_diverse_7_beam_search,R. Kelly has released collaboration albums wit...,0.996436,0.998380,0.874548,True,False,True,NaN,NaN,NaN,NaN,NaN,79.858452,-1
3,6140,t5_small_diverse_7_beam_search,R. Kelly released collaboration albums with Ja...,0.996924,0.997682,0.875569,True,False,True,NaN,NaN,NaN,NaN,NaN,117.868538,1
4,6140,t5_small_diverse_7_beam_search,R. Kelly is one of the best-selling music arti...,0.994441,0.994815,0.879582,True,True,True,NaN,NaN,NaN,NaN,NaN,16.624537,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15555,3541,t5_small_multiclaim,Adrianne Palicki was born in 1983.,0.997649,0.999215,0.892769,True,True,True,3311.0,1.0,1.0,1.0,1.0,18.963940,-1
15556,3541,t5_small_multiclaim,Adrianne Palicki is an American.,0.999051,0.999498,0.902723,True,True,True,3311.0,1.0,1.0,1.0,1.0,23.555801,-1
15557,3541,t5_small_multiclaim,Adrianne Palicki is best known for her role as...,0.998119,0.995047,0.896891,False,True,True,3311.0,1.0,1.0,1.0,1.0,19.013100,-1
15558,3541,t5_small_multiclaim,Adrianne Palicki is best known for her role in...,0.165706,0.313146,0.898684,False,True,True,1311.0,0.0,1.0,1.0,1.0,16.101629,-1


In [24]:
df2.to_json("data/claim-extraction-acl24/single_metrics_all.jsonl", orient="records", lines=True)

In [4]:
df.describe()

,sentence_id,F,F_lg,F_bert,annotation,F_gold,G_gold,D_gold,A_gold,pplxt_generated,pplxt_generated_coedit_l,pplxt_generated_coedit_xl,bpeslor,scribendi_score_l,scribendi_score_xl,jaccard_bigram_l,jaccard_bigram_xl,levenshtein_l,levenshtein_xl
count,15560.000000,15560.000000,15560.000000,15560.000000,1375.000000,1375.000000,1375.000000,1375.000000,1375.000000,15560.000000,15560.000000,15560.000000,15560.000000,15560.000000,15560.000000,15560.000000,15560.000000,15560.000000,15560.000000
mean,3168.064974,0.920670,0.923850,0.886349,3036.782545,0.840000,0.905455,0.939636,0.986182,116.603742,93.365327,82.396951,4.217108,-0.189332,-0.097429,0.533176,0.532756,18.366902,17.402763
std,1899.435757,0.211934,0.225887,0.021105,645.575673,0.366739,0.292693,0.238246,0.116778,303.729960,632.427546,188.310414,1.008419,0.849957,0.874116,0.330925,0.324268,26.591801,35.524083
min,4.000000,0.000340,0.000200,0.720988,111.000000,0.000000,0.000000,0.000000,0.000000,4.620808,1.274006,1.401227,-1.134594,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000
25%,1475.000000,0.977839,0.987401,0.874214,3301.000000,1.000000,1.000000,1.000000,1.000000,33.150960,27.051923,25.799359,3.577769,-1.000000,-1.000000,0.250000,0.265789,1.000000,3.000000
50%,3150.000000,0.993456,0.995178,0.888115,3311.000000,1.000000,1.000000,1.000000,1.000000,58.956457,47.646042,45.524208,4.260179,0.000000,0.000000,0.500000,0.500000,12.000000,12.000000
75%,4831.000000,0.996740,0.998083,0.900338,3311.000000,1.000000,1.000000,1.000000,1.000000,114.328327,90.940073,84.447382,4.901421,1.000000,1.000000,0.858766,0.800000,27.000000,25.000000
max,6671.000000,0.999175,0.999623,0.954520,3311.000000,1.000000,1.000000,1.000000,1.000000,24376.843750,75713.343750,11399.097656,7.995834,1.000000,1.000000,1.000000,1.000000,1518.000000,1689.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15560 entries, 0 to 15559
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   sentence_id                15560 non-null  int64  
 1   model                      15560 non-null  object 
 2   generated                  15560 non-null  object 
 3   F                          15560 non-null  float64
 4   F_lg                       15560 non-null  float64
 5   F_bert                     15560 non-null  float64
 6   D                          15560 non-null  bool   
 7   A                          15560 non-null  bool   
 8   A_luke                     15560 non-null  bool   
 9   annotation                 1375 non-null   float64
 10  F_gold                     1375 non-null   float64
 11  G_gold                     1375 non-null   float64
 12  D_gold                     1375 non-null   float64
 13  A_gold                     1375 non-null   flo

In [4]:
multi_df = pd.read_json("/mnt/data/factcheck/_acl24/multi_metrics.jsonl", lines=True, dtype={"focus_annotation": str, "coverage_annotation":str})
multi_df.head()

,sentence_id,model,generated,Foc,Cov,Red,focus_annotation,coverage_annotation,Foc_gold,Cov_gold,Foc_claimwise,Cov_claimwise,Foc_qags,Cov_qags,Foc_nli,Cov_nli
0,6140,t5_small_diverse_7_beam_search,"[R. Kelly is a music artist., R. Kelly has bee...",0.422021,0.655813,0.558942,None,None,NaN,NaN,"[0.9876041412000001, 0.0008317942, 0.001073256...","[0.0013360368000000001, 0.9885496497, 0.977553...",0.307692,0.500000,0.426096,0.663002
1,4799,t5_small_diverse_7_beam_search,[Greenpeace is a non-governmental environmenta...,0.036881,0.285785,0.731872,None,None,NaN,NaN,"[0.0733137056, 0.0244704001, 0.0223413687, 0.0...","[0.000407305, 0.4388080835, 0.9888217449000001...",0.444444,0.200000,0.026631,0.368504
2,4773,t5_small_diverse_7_beam_search,[Girls' Generation is a South Korean girl grou...,0.872058,0.661790,0.671629,None,None,NaN,NaN,"[0.9809049368, 0.9545260668000001, 0.958105266...","[0.9793230295, 0.9175070524000001, 0.487090945...",0.777778,0.666667,0.792251,0.594111
3,617,t5_small_diverse_7_beam_search,"[Lisa Lopes was a rapper., Lisa Lopes was an a...",0.018666,0.002991,0.544763,None,None,NaN,NaN,"[0.0024146722, 0.0859003961, 0.0144883571, 0.0...","[0.0017014290000000002, 0.0015977740000000002,...",0.000000,0.000000,0.247040,0.007097
4,5387,t5_small_diverse_7_beam_search,"[Linkin Park is a rock band., Linkin Park has ...",0.181419,0.988288,0.168023,None,None,NaN,NaN,"[0.0634528026, 0.9958975315, 0.0087414458, 0.0...","[0.9868214726000001, 0.9897550941000001]",0.307692,0.666667,0.302033,0.992849


## 📏 Metric Validation

get gold data only dataframes

In [3]:
gold_df = df.dropna()
gold_df

,sentence_id,model,generated,F,F_lg,F_bert,D,A,A_luke,annotation,...,pplxt_generated_coedit_xl,identity_l,identity_xl,bpeslor,scribendi_score_l,scribendi_score_xl,jaccard_bigram_l,jaccard_bigram_xl,levenshtein_l,levenshtein_xl
106,2136,t5_small_diverse_7_beam_search,Lisa Lopes won four Grammy Awards.,0.997643,0.994213,0.917318,False,True,True,3311.0,...,101.775452,True,False,4.495110,0,1,1.000000,0.571429,0,4
107,2136,t5_small_diverse_7_beam_search,Lisa Lopes has won four Grammy Awards.,0.996267,0.994565,0.915941,True,True,True,3311.0,...,101.775452,True,True,4.710234,0,0,1.000000,1.000000,0,0
108,2136,t5_small_diverse_7_beam_search,Lisa Lopes worked with TLC.,0.997767,0.999344,0.915106,False,True,True,3311.0,...,157.765091,True,False,3.149786,0,-1,1.000000,0.222222,0,14
109,2136,t5_small_diverse_7_beam_search,Lisa Lopes is an actress.,0.002128,0.003270,0.901596,True,True,True,2311.0,...,55.572781,True,False,3.850367,0,1,1.000000,0.500000,0,9
110,2136,t5_small_diverse_7_beam_search,Lisa Lopes was in TLC.,0.997126,0.998950,0.909543,True,True,True,2311.0,...,176.713043,True,False,2.436710,0,1,1.000000,0.333333,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15555,3541,t5_small_multiclaim,Adrianne Palicki was born in 1983.,0.997649,0.999215,0.892769,True,True,True,3311.0,...,41.039127,False,False,5.651093,-1,-1,0.500000,0.500000,15,15
15556,3541,t5_small_multiclaim,Adrianne Palicki is an American.,0.999051,0.999498,0.902723,True,True,True,3311.0,...,18.124359,False,False,5.085773,-1,1,0.176471,0.500000,64,6
15557,3541,t5_small_multiclaim,Adrianne Palicki is best known for her role as...,0.998119,0.995047,0.896891,False,True,True,3311.0,...,50.003304,False,False,5.651744,-1,-1,0.727273,0.500000,15,20
15558,3541,t5_small_multiclaim,Adrianne Palicki is best known for her role in...,0.165706,0.313146,0.898684,False,True,True,1311.0,...,47.368946,False,False,5.373777,-1,-1,0.700000,0.454545,15,20


In [5]:
multi_df_gold = multi_df.dropna()
multi_df_gold.head()

,sentence_id,model,generated,Foc,Cov,Red,focus_annotation,coverage_annotation,Foc_gold,Cov_gold,Foc_claimwise,Cov_claimwise,Foc_qags,Cov_qags,Foc_nli,Cov_nli
18,2136,t5_small_diverse_7_beam_search,"[Lisa Lopes won four Grammy Awards., Lisa Lope...",0.558724,0.527328,0.899433,11001,0001,0.600000,0.25,"[0.000699135, 0.0007011201, 0.9724544287, 0.85...","[0.1559358388, 0.9746372104000001, 0.005459909...",0.436364,0.485714,0.413961,0.486494
31,2419,t5_small_diverse_7_beam_search,[The jackfruit is native to parts of South and...,0.732900,0.496445,0.738563,1111,11011,1.000000,0.80,"[0.6799927354, 0.6455110908, 0.6699844599, 0.9...","[0.570248127, 0.009709138400000001, 0.00135349...",0.750000,0.833333,0.926292,0.754926
44,5868,t5_small_diverse_7_beam_search,[Blue whales were hunted almost to extinction....,0.743895,0.975211,0.727786,1111,11,1.000000,1.00,"[0.991527915, 0.9913563728, 0.0017519878, 0.99...","[0.9768199325, 0.9736022949]",0.000000,0.000000,0.009115,0.997859
47,3115,t5_small_diverse_7_beam_search,[Muammar Gaddafi was Chairperson of the Africa...,0.642387,0.467188,0.471180,1011010,0011,0.571429,0.50,"[0.9904093742000001, 0.0237617083, 0.168916568...","[0.0178864598, 0.0276789367, 0.8470592499, 0.9...",0.897436,0.520833,0.595558,0.251312
53,3809,t5_small_diverse_7_beam_search,[Prague is the 14th largest city in the Europe...,0.616960,0.307503,0.477416,01111,01010,0.800000,0.40,"[0.0018640353, 0.9733234644000001, 0.985452473...","[0.0007203866, 0.5078630447, 0.051747359300000...",0.300000,0.444444,0.788377,0.377059


### Faithfulness

F, F_lg, F_bert

must be discretized on train -> eval on test

In [17]:
def eval_faithfulness(metric_name: str, threshold=None):
    if threshold is None:
        train_f_gold, test_f_gold, train_f, test_f = train_test_split(gold_df["F_gold"].to_numpy(), gold_df[metric_name].to_numpy(), test_size=0.9, random_state=SEED)

        def opt_fun(threshold):
            ypred = train_f >= threshold
            return -metrics.f1_score(y_true=train_f_gold, y_pred=ypred)

        res = opt.minimize_scalar(opt_fun, bounds=(0,1), method="bounded")
        f1_train = -res.fun
        threshold = res.x
        f1_test = metrics.f1_score(test_f_gold, test_f >= threshold)
    else:
        f1_train = None
        f1_test = metrics.f1_score(gold_df["F_gold"].to_numpy(), gold_df[metric_name].to_numpy() >= threshold)

    auc = metrics.roc_auc_score(gold_df["F_gold"].to_numpy(), gold_df[metric_name].to_numpy())

    return f1_train, f1_test, threshold, auc
    

In [18]:
for metric in ["F", "F_lg", "F_bert"]:
    f1_train, f1_test, threshold, auc = eval_faithfulness(metric, threshold=0.5)
    print("Metric name:", metric)
    print("f1_train", f1_train, "\nf1_test", f1_test, "\nthreshold", threshold, "\nauc", auc)
    print()

Metric name: F
f1_train None 
f1_test 0.9209340434248259 
threshold 0.5 
auc 0.7988488783943328

Metric name: F_lg
f1_train None 
f1_test 0.9325153374233128 
threshold 0.5 
auc 0.8249586776859504

Metric name: F_bert
f1_train None 
f1_test 0.9130434782608696 
threshold 0.5 
auc 0.6064895710350257



### Fluency

binarize (those where possible)

-> Scribendi score, identity

In [10]:
def eval_scribendi(metric_name):
    bin_scribendi = gold_df[metric_name].to_numpy() != 1
    return metrics.f1_score(gold_df["G_gold"].to_numpy(), bin_scribendi)

def eval_identity(metric_name):
    return metrics.f1_score(gold_df["G_gold"].to_numpy(), gold_df[metric_name].to_numpy())

def eval_fluency(metric_name):
    #K-fold crossvalidation
    
    gold_data = gold_df["G_gold"].to_numpy()
    metric_data = gold_df[metric_name].to_numpy()

    kf = KFold(50, shuffle=True, random_state=SEED)
    gold_data_train, gold_data_test, metric_data_train, metric_data_test = train_test_split(gold_data, metric_data, test_size=0.9, random_state=SEED)

    test_results = []
    train_results = []
    thresholds = []
    for train_index, test_index in kf.split(gold_data_train):
        train_f_gold = gold_data_train[train_index]
        test_f_gold = gold_data_train[test_index]
        train_f = metric_data_train[train_index]
        test_f = metric_data_train[test_index]

        def opt_fun(threshold):
            ypred = train_f >= threshold
            return -metrics.f1_score(y_true=train_f_gold, y_pred=ypred)

        res = opt.minimize_scalar(opt_fun, bounds=(np.min(train_f),np.max(train_f)), method="bounded")

        tmp_f1_train = -res.fun
        train_results.append(tmp_f1_train)

        tmp_threshold = res.x
        thresholds.append(tmp_threshold)


        tmp_f1_test = metrics.f1_score(test_f_gold, test_f >= tmp_threshold)
        test_results.append(tmp_f1_test)

    f1_val = np.average(test_results)
    f1_train = np.average(train_results)
    threshold = np.average(thresholds)
    f1_test = metrics.f1_score(gold_data_test, metric_data_test >= threshold)

    return f1_train, f1_test, threshold, f1_val



In [9]:
#METRICS = ["pplxt_generated", "identity_l","identity_xl", "bpeslor", "scribendi_score_l", "scribendi_score_xl", "jaccard_bigram_l", "jaccard_bigram_xl", "jaccard_bigram_regex_xl", "jaccard_bigram_regex_l", "levenshtein_l", "levenshtein_xl", "levenshtein_norm_l", "levenshtein_norm_xl"]
METRICS = ["pplxt_generated", "scribendi_score_l", "scribendi_score_xl", "perplexity_compare_l", "perplexity_compare_xl"]

In [12]:
for metric in tqdm(METRICS):
    if metric == "scribendi_score_l" or metric == "scribendi_score_xl" or metric == "perplexity_compare_l" or metric == "perplexity_compare_xl":
        res = eval_scribendi(metric)
        print("metric_name:", metric)
        print("f1 score:", res)
        print()
    elif metric == "identity_l" or metric == "identity_xl":
        res = eval_identity(metric)
        print("metric_name:", metric)
        print("f1 score:", res)
        print()
    else:
        f1_train, f1_test, threshold, f1_val = eval_fluency(metric)
        print("Metric name:", metric)
        print("f1_train", f1_train, "\nf1_test", f1_test, "\nthreshold", threshold, "\nf1_val", f1_val)
        print()

100%|██████████| 5/5 [00:02<00:00,  2.09it/s]

Metric name: pplxt_generated
f1_train 0.9417095167457354 
f1_test 0.9445865302642795 
threshold 10.366122618981287 
f1_val 0.9273333333333335

metric_name: scribendi_score_l
f1 score: 0.7998196573489631

metric_name: scribendi_score_xl
f1 score: 0.7416587225929456

metric_name: perplexity_compare_l
f1 score: 0.38839848675914257

metric_name: perplexity_compare_xl
f1 score: 0.3148765843895931



Metric name: pplxt_generated
f1_train 0.9416342412451362 
f1_test 0.9409751924721983 
threshold 10.679540702289778

metric_name: identity_l
f1 score: 0.456865127582017

metric_name: identity_xl
f1 score: 0.394224733207784

Metric name: bpeslor
f1_train 0.9279999999999999 
f1_test 0.9163346613545817 
threshold 2.884907699039611

metric_name: scribendi_score_l
f1 score: 0.7998196573489631

metric_name: scribendi_score_xl
f1 score: 0.7416587225929456

Metric name: jaccard_bigram_l
f1_train 0.9375 
f1_test 0.9361518550474547 
threshold 0.026774040609635223

Metric name: jaccard_bigram_xl
f1_train 0.9169960474308301 
f1_test 0.9352331606217616 
threshold 0.06558138190048653

Metric name: levenshtein_l
f1_train 0.7706422018348624 
f1_test 0.7846076961519239 
threshold 0.5354608799840744

Metric name: levenshtein_xl
f1_train 0.7798165137614679 
f1_test 0.8131760078662732 
threshold 2.32867939930306



### Decontextualization

In [21]:
#Decontextualization
metrics.f1_score(gold_df["D_gold"], gold_df["D"])

0.8581138487680544

### Atomicity

In [22]:
#f1 A
metrics.f1_score(gold_df["A_gold"], gold_df["A"])

0.9566210045662101

In [23]:
#f1 A luke
metrics.f1_score(gold_df["A_gold"], gold_df["A_luke"])

0.9432651485912775

### Focus

In [9]:
multi_df_gold

,sentence_id,model,generated,Foc,Cov,Red,focus_annotation,coverage_annotation,Foc_gold,Cov_gold,Foc_claimwise,Cov_claimwise,Foc_qags,Cov_qags,Foc_nli,Cov_nli
18,2136,t5_small_diverse_7_beam_search,"[Lisa Lopes won four Grammy Awards., Lisa Lope...",0.558724,0.527328,0.899433,11001,0001,0.600000,0.25,"[0.000699135, 0.0007011201, 0.9724544287, 0.85...","[0.1559358388, 0.9746372104000001, 0.005459909...",0.436364,0.485714,0.413961,0.486494
31,2419,t5_small_diverse_7_beam_search,[The jackfruit is native to parts of South and...,0.732900,0.496445,0.738563,1111,11011,1.000000,0.80,"[0.6799927354, 0.6455110908, 0.6699844599, 0.9...","[0.570248127, 0.009709138400000001, 0.00135349...",0.750000,0.833333,0.926292,0.754926
44,5868,t5_small_diverse_7_beam_search,[Blue whales were hunted almost to extinction....,0.743895,0.975211,0.727786,1111,11,1.000000,1.00,"[0.991527915, 0.9913563728, 0.0017519878, 0.99...","[0.9768199325, 0.9736022949]",0.000000,0.000000,0.009115,0.997859
47,3115,t5_small_diverse_7_beam_search,[Muammar Gaddafi was Chairperson of the Africa...,0.642387,0.467188,0.471180,1011010,0011,0.571429,0.50,"[0.9904093742000001, 0.0237617083, 0.168916568...","[0.0178864598, 0.0276789367, 0.8470592499, 0.9...",0.897436,0.520833,0.595558,0.251312
53,3809,t5_small_diverse_7_beam_search,[Prague is the 14th largest city in the Europe...,0.616960,0.307503,0.477416,01111,01010,0.800000,0.40,"[0.0018640353, 0.9733234644000001, 0.985452473...","[0.0007203866, 0.5078630447, 0.051747359300000...",0.300000,0.444444,0.788377,0.377059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2169,2571,t5_small_multiclaim,[Ecuador's ethnically diverse population is fo...,0.427916,0.562199,0.260103,0000,01100,0.000000,0.40,"[0.0044027194, 0.9418351054, 0.001285269700000...","[0.9543712139, 0.09293915330000001, 0.99613463...",0.571429,0.333333,0.251081,0.784492
2177,1441,t5_small_multiclaim,"[Honey to the B is by Billie Piper., Honey to ...",0.962540,0.018907,0.812428,11101,0000,0.800000,0.00,"[0.9712346196, 0.9817345142, 0.9712346196, 0.9...","[0.0583052821, 0.0018448567, 0.0064986916, 0.0...",0.750000,0.600000,0.989503,0.000345
2200,1298,t5_small_multiclaim,[Richard Ramirez terrorized the residents of t...,0.473229,0.449771,0.496719,10000,01000,0.200000,0.20,"[0.8644894361000001, 0.3660726845, 0.739655196...","[0.9118559361, 0.9015911818000001, 0.002224959...",0.500000,0.370370,0.587031,0.423775
2215,6003,t5_small_multiclaim,[Angela Lansbury contributed to Disney's Beaut...,0.332535,0.527549,0.796507,010,10,0.333333,0.50,"[0.0018639604000000001, 0.9938004613, 0.001941...","[0.9853401780000001, 0.06975777450000001]",0.166667,1.000000,0.332582,0.975697


In [10]:
multi_df_gold["focus_annotation"] = multi_df_gold["focus_annotation"].apply(lambda x: [int(digit) for digit in x])

/tmp/ipykernel_1611376/2089343963.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  multi_df_gold["focus_annotation"] = multi_df_gold["focus_annotation"].apply(lambda x: [int(digit) for digit in x])


In [11]:
#there are two rows where focus is annotated with some mistake (more annotated than evaluated)

def compare_lens(row):
    return len(row["focus_annotation"]) != len(row["Foc_claimwise"])

multi_df_gold[multi_df_gold.apply(compare_lens, axis=1)]

,sentence_id,model,generated,Foc,Cov,Red,focus_annotation,coverage_annotation,Foc_gold,Cov_gold,Foc_claimwise,Cov_claimwise,Foc_qags,Cov_qags,Foc_nli,Cov_nli
634,1834,qlora-mistral-instruct-v0.2,[Prometheus's leading actors had roles in tele...,0.023599,0.140397,0.241546,"[0, 0, 0, 0, 0, 0, 0]",00,0.0,0.0,"[0.0112357512, 0.0200793613, 0.062200896400000...","[0.27893912790000003, 0.001855043]",0.25,0.333333,0.002141,0.487191
1327,6003,qacg,[Angela Lansbury co-owned Corymore Productions...,0.003301,0.460686,0.387945,"[0, 0, 0, 0, 0, 0]",00,0.0,0.0,"[0.0013734545, 0.0018639604000000001, 0.000766...","[0.9115316868000001, 0.0098411804]",0.00,0.500000,0.000256,0.496214


In [12]:
#compute brier scores
brier_scores = np.array([metrics.brier_score_loss(true, pred_proba) if len(true) == len(pred_proba) else np.nan for true, pred_proba in zip(multi_df_gold["focus_annotation"], multi_df_gold["Foc_claimwise"])], dtype=np.float64)
brier_scores

array([7.34051191e-01, 8.52647504e-02, 2.49181891e-01, 2.48577356e-01,
       1.47306301e-01, 2.49001099e-04, 1.18508282e-01, 5.21013529e-01,
       2.58457752e-01, 3.94140514e-01, 5.57849101e-03, 4.83738075e-04,
       1.97664886e-01, 1.45414662e-01, 1.55881741e-01, 2.32939314e-01,
       1.64420972e-04, 5.10505090e-02, 2.55328443e-01, 5.41679593e-02,
       5.23411795e-01, 1.10042159e-03, 5.97604723e-02, 4.04806606e-04,
       3.57324423e-03, 5.30123533e-01, 3.54796954e-01, 1.52625827e-04,
       5.44070744e-02, 5.97549646e-01, 2.47996670e-01, 9.43773894e-02,
       5.64232436e-01, 1.34449304e-01, 2.08247961e-01, 3.97590631e-01,
       6.67214847e-02, 3.67808993e-01, 2.85791092e-01, 1.33439826e-02,
       9.72134917e-01, 6.69584334e-02, 2.45341733e-01, 5.58618055e-02,
       9.00776762e-04, 1.28235873e-01, 2.98107441e-01, 1.51868024e-01,
       2.06766270e-01, 8.44326837e-01, 6.33308596e-01, 4.72144803e-04,
       1.05268301e-03, 2.08838212e-01, 2.03172616e-01, 1.62799259e-01,
      

In [13]:
#get indices of nan values in brier scores
nan_indices = np.argwhere(np.isnan(brier_scores)).flatten()
nan_indices

array([ 56, 117])

In [14]:
#explore the nan rows -> the same rows as before
multi_df_gold.iloc[nan_indices]

,sentence_id,model,generated,Foc,Cov,Red,focus_annotation,coverage_annotation,Foc_gold,Cov_gold,Foc_claimwise,Cov_claimwise,Foc_qags,Cov_qags,Foc_nli,Cov_nli
634,1834,qlora-mistral-instruct-v0.2,[Prometheus's leading actors had roles in tele...,0.023599,0.140397,0.241546,"[0, 0, 0, 0, 0, 0, 0]",00,0.0,0.0,"[0.0112357512, 0.0200793613, 0.062200896400000...","[0.27893912790000003, 0.001855043]",0.25,0.333333,0.002141,0.487191
1327,6003,qacg,[Angela Lansbury co-owned Corymore Productions...,0.003301,0.460686,0.387945,"[0, 0, 0, 0, 0, 0]",00,0.0,0.0,"[0.0013734545, 0.0018639604000000001, 0.000766...","[0.9115316868000001, 0.0098411804]",0.00,0.500000,0.000256,0.496214


In [15]:
#result brier_scores focus
np.nanmean(brier_scores)

0.1507313343128393

In [18]:
mse_scores = np.array([metrics.mean_squared_error(true, pred_proba) if len(true) == len(pred_proba) else np.nan for true, pred_proba in zip(multi_df_gold["focus_annotation"], multi_df_gold["Foc_claimwise"])], dtype=np.float64)
print(np.nanmean(mse_scores))

0.1507313343128393


In [20]:
rmse_scores = np.array([metrics.root_mean_squared_error(true, pred_proba) if len(true) == len(pred_proba) else np.nan for true, pred_proba in zip(multi_df_gold["focus_annotation"], multi_df_gold["Foc_claimwise"])], dtype=np.float64)
print(np.nanmean(rmse_scores))

0.30189479555369975


In [21]:
#RMSE Foc gold X Foc
rmse = metrics.root_mean_squared_error(multi_df_gold["Foc_gold"], multi_df_gold["Foc"])
print(rmse)

#MSE Foc gold X Foc
mse = metrics.mean_squared_error(multi_df_gold["Foc_gold"], multi_df_gold["Foc"])
print(mse)

0.2250816317894012
0.050661740968979574


### Coverage

In [24]:
multi_df_gold["coverage_annotation"] = multi_df_gold["coverage_annotation"].apply(lambda x: [int(digit) for digit in x])
multi_df_gold

/tmp/ipykernel_1611376/390246212.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  multi_df_gold["coverage_annotation"] = multi_df_gold["coverage_annotation"].apply(lambda x: [int(digit) for digit in x])


,sentence_id,model,generated,Foc,Cov,Red,focus_annotation,coverage_annotation,Foc_gold,Cov_gold,Foc_claimwise,Cov_claimwise,Foc_qags,Cov_qags,Foc_nli,Cov_nli
18,2136,t5_small_diverse_7_beam_search,"[Lisa Lopes won four Grammy Awards., Lisa Lope...",0.558724,0.527328,0.899433,"[1, 1, 0, 0, 1]","[0, 0, 0, 1]",0.600000,0.25,"[0.000699135, 0.0007011201, 0.9724544287, 0.85...","[0.1559358388, 0.9746372104000001, 0.005459909...",0.436364,0.485714,0.413961,0.486494
31,2419,t5_small_diverse_7_beam_search,[The jackfruit is native to parts of South and...,0.732900,0.496445,0.738563,"[1, 1, 1, 1]","[1, 1, 0, 1, 1]",1.000000,0.80,"[0.6799927354, 0.6455110908, 0.6699844599, 0.9...","[0.570248127, 0.009709138400000001, 0.00135349...",0.750000,0.833333,0.926292,0.754926
44,5868,t5_small_diverse_7_beam_search,[Blue whales were hunted almost to extinction....,0.743895,0.975211,0.727786,"[1, 1, 1, 1]","[1, 1]",1.000000,1.00,"[0.991527915, 0.9913563728, 0.0017519878, 0.99...","[0.9768199325, 0.9736022949]",0.000000,0.000000,0.009115,0.997859
47,3115,t5_small_diverse_7_beam_search,[Muammar Gaddafi was Chairperson of the Africa...,0.642387,0.467188,0.471180,"[1, 0, 1, 1, 0, 1, 0]","[0, 0, 1, 1]",0.571429,0.50,"[0.9904093742000001, 0.0237617083, 0.168916568...","[0.0178864598, 0.0276789367, 0.8470592499, 0.9...",0.897436,0.520833,0.595558,0.251312
53,3809,t5_small_diverse_7_beam_search,[Prague is the 14th largest city in the Europe...,0.616960,0.307503,0.477416,"[0, 1, 1, 1, 1]","[0, 1, 0, 1, 0]",0.800000,0.40,"[0.0018640353, 0.9733234644000001, 0.985452473...","[0.0007203866, 0.5078630447, 0.051747359300000...",0.300000,0.444444,0.788377,0.377059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2169,2571,t5_small_multiclaim,[Ecuador's ethnically diverse population is fo...,0.427916,0.562199,0.260103,"[0, 0, 0, 0]","[0, 1, 1, 0, 0]",0.000000,0.40,"[0.0044027194, 0.9418351054, 0.001285269700000...","[0.9543712139, 0.09293915330000001, 0.99613463...",0.571429,0.333333,0.251081,0.784492
2177,1441,t5_small_multiclaim,"[Honey to the B is by Billie Piper., Honey to ...",0.962540,0.018907,0.812428,"[1, 1, 1, 0, 1]","[0, 0, 0, 0]",0.800000,0.00,"[0.9712346196, 0.9817345142, 0.9712346196, 0.9...","[0.0583052821, 0.0018448567, 0.0064986916, 0.0...",0.750000,0.600000,0.989503,0.000345
2200,1298,t5_small_multiclaim,[Richard Ramirez terrorized the residents of t...,0.473229,0.449771,0.496719,"[1, 0, 0, 0, 0]","[0, 1, 0, 0, 0]",0.200000,0.20,"[0.8644894361000001, 0.3660726845, 0.739655196...","[0.9118559361, 0.9015911818000001, 0.002224959...",0.500000,0.370370,0.587031,0.423775
2215,6003,t5_small_multiclaim,[Angela Lansbury contributed to Disney's Beaut...,0.332535,0.527549,0.796507,"[0, 1, 0]","[1, 0]",0.333333,0.50,"[0.0018639604000000001, 0.9938004613, 0.001941...","[0.9853401780000001, 0.06975777450000001]",0.166667,1.000000,0.332582,0.975697


In [25]:
brier_scores_coverage = np.array([metrics.brier_score_loss(true, pred) if len(true) == len(pred) else np.nan for true, pred in zip(multi_df_gold["coverage_annotation"], multi_df_gold["Cov_claimwise"])], dtype=np.float64)

In [26]:
#brier score mean coverage
np.nanmean(brier_scores_coverage)

0.1154968152790911

In [29]:
#RMSE Cov gold X Cov
print(metrics.root_mean_squared_error(multi_df_gold["Cov_gold"], multi_df_gold["Cov"]))

#MSE Cov gold X Cov
print(metrics.mean_squared_error(multi_df_gold["Cov_gold"], multi_df_gold["Cov"]))

0.22264261282395167
0.04956973304507605


### F_fact

In [6]:
multi_df_gold

,sentence_id,model,generated,Foc,Cov,Red,focus_annotation,coverage_annotation,Foc_gold,Cov_gold,Foc_claimwise,Cov_claimwise,Foc_qags,Cov_qags,Foc_nli,Cov_nli
18,2136,t5_small_diverse_7_beam_search,"[Lisa Lopes won four Grammy Awards., Lisa Lope...",0.558724,0.527328,0.899433,11001,0001,0.600000,0.25,"[0.000699135, 0.0007011201, 0.9724544287, 0.85...","[0.1559358388, 0.9746372104000001, 0.005459909...",0.436364,0.485714,0.413961,0.486494
31,2419,t5_small_diverse_7_beam_search,[The jackfruit is native to parts of South and...,0.732900,0.496445,0.738563,1111,11011,1.000000,0.80,"[0.6799927354, 0.6455110908, 0.6699844599, 0.9...","[0.570248127, 0.009709138400000001, 0.00135349...",0.750000,0.833333,0.926292,0.754926
44,5868,t5_small_diverse_7_beam_search,[Blue whales were hunted almost to extinction....,0.743895,0.975211,0.727786,1111,11,1.000000,1.00,"[0.991527915, 0.9913563728, 0.0017519878, 0.99...","[0.9768199325, 0.9736022949]",0.000000,0.000000,0.009115,0.997859
47,3115,t5_small_diverse_7_beam_search,[Muammar Gaddafi was Chairperson of the Africa...,0.642387,0.467188,0.471180,1011010,0011,0.571429,0.50,"[0.9904093742000001, 0.0237617083, 0.168916568...","[0.0178864598, 0.0276789367, 0.8470592499, 0.9...",0.897436,0.520833,0.595558,0.251312
53,3809,t5_small_diverse_7_beam_search,[Prague is the 14th largest city in the Europe...,0.616960,0.307503,0.477416,01111,01010,0.800000,0.40,"[0.0018640353, 0.9733234644000001, 0.985452473...","[0.0007203866, 0.5078630447, 0.051747359300000...",0.300000,0.444444,0.788377,0.377059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2169,2571,t5_small_multiclaim,[Ecuador's ethnically diverse population is fo...,0.427916,0.562199,0.260103,0000,01100,0.000000,0.40,"[0.0044027194, 0.9418351054, 0.001285269700000...","[0.9543712139, 0.09293915330000001, 0.99613463...",0.571429,0.333333,0.251081,0.784492
2177,1441,t5_small_multiclaim,"[Honey to the B is by Billie Piper., Honey to ...",0.962540,0.018907,0.812428,11101,0000,0.800000,0.00,"[0.9712346196, 0.9817345142, 0.9712346196, 0.9...","[0.0583052821, 0.0018448567, 0.0064986916, 0.0...",0.750000,0.600000,0.989503,0.000345
2200,1298,t5_small_multiclaim,[Richard Ramirez terrorized the residents of t...,0.473229,0.449771,0.496719,10000,01000,0.200000,0.20,"[0.8644894361000001, 0.3660726845, 0.739655196...","[0.9118559361, 0.9015911818000001, 0.002224959...",0.500000,0.370370,0.587031,0.423775
2215,6003,t5_small_multiclaim,[Angela Lansbury contributed to Disney's Beaut...,0.332535,0.527549,0.796507,010,10,0.333333,0.50,"[0.0018639604000000001, 0.9938004613, 0.001941...","[0.9853401780000001, 0.06975777450000001]",0.166667,1.000000,0.332582,0.975697


In [11]:
multi_df_gold["f_fact_gold"] = 2*(multi_df_gold["Foc_gold"]*multi_df_gold["Cov_gold"])/(multi_df_gold["Foc"]+multi_df_gold["Cov"])
multi_df_gold["f_fact"] = 2*(multi_df_gold["Foc"]*multi_df_gold["Cov"])/(multi_df_gold["Foc"]+multi_df_gold["Cov"])
multi_df_gold

/tmp/ipykernel_1713571/1955912386.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  multi_df_gold["f_fact_gold"] = 2*(multi_df_gold["Foc_gold"]*multi_df_gold["Cov_gold"])/(multi_df_gold["Foc"]+multi_df_gold["Cov"])
/tmp/ipykernel_1713571/1955912386.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  multi_df_gold["f_fact"] = 2*(multi_df_gold["Foc"]*multi_df_gold["Cov"])/(multi_df_gold["Foc"]+multi_df_gold["Cov"])


,sentence_id,model,generated,Foc,Cov,Red,focus_annotation,coverage_annotation,Foc_gold,Cov_gold,Foc_claimwise,Cov_claimwise,Foc_qags,Cov_qags,Foc_nli,Cov_nli,f_fact_gold,f_fact
18,2136,t5_small_diverse_7_beam_search,"[Lisa Lopes won four Grammy Awards., Lisa Lope...",0.558724,0.527328,0.899433,11001,0001,0.600000,0.25,"[0.000699135, 0.0007011201, 0.9724544287, 0.85...","[0.1559358388, 0.9746372104000001, 0.005459909...",0.436364,0.485714,0.413961,0.486494,0.276230,0.542572
31,2419,t5_small_diverse_7_beam_search,[The jackfruit is native to parts of South and...,0.732900,0.496445,0.738563,1111,11011,1.000000,0.80,"[0.6799927354, 0.6455110908, 0.6699844599, 0.9...","[0.570248127, 0.009709138400000001, 0.00135349...",0.750000,0.833333,0.926292,0.754926,1.301507,0.591932
44,5868,t5_small_diverse_7_beam_search,[Blue whales were hunted almost to extinction....,0.743895,0.975211,0.727786,1111,11,1.000000,1.00,"[0.991527915, 0.9913563728, 0.0017519878, 0.99...","[0.9768199325, 0.9736022949]",0.000000,0.000000,0.009115,0.997859,1.163395,0.843991
47,3115,t5_small_diverse_7_beam_search,[Muammar Gaddafi was Chairperson of the Africa...,0.642387,0.467188,0.471180,1011010,0011,0.571429,0.50,"[0.9904093742000001, 0.0237617083, 0.168916568...","[0.0178864598, 0.0276789367, 0.8470592499, 0.9...",0.897436,0.520833,0.595558,0.251312,0.514998,0.540956
53,3809,t5_small_diverse_7_beam_search,[Prague is the 14th largest city in the Europe...,0.616960,0.307503,0.477416,01111,01010,0.800000,0.40,"[0.0018640353, 0.9733234644000001, 0.985452473...","[0.0007203866, 0.5078630447, 0.051747359300000...",0.300000,0.444444,0.788377,0.377059,0.692294,0.410437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2169,2571,t5_small_multiclaim,[Ecuador's ethnically diverse population is fo...,0.427916,0.562199,0.260103,0000,01100,0.000000,0.40,"[0.0044027194, 0.9418351054, 0.001285269700000...","[0.9543712139, 0.09293915330000001, 0.99613463...",0.571429,0.333333,0.251081,0.784492,0.000000,0.485952
2177,1441,t5_small_multiclaim,"[Honey to the B is by Billie Piper., Honey to ...",0.962540,0.018907,0.812428,11101,0000,0.800000,0.00,"[0.9712346196, 0.9817345142, 0.9712346196, 0.9...","[0.0583052821, 0.0018448567, 0.0064986916, 0.0...",0.750000,0.600000,0.989503,0.000345,0.000000,0.037086
2200,1298,t5_small_multiclaim,[Richard Ramirez terrorized the residents of t...,0.473229,0.449771,0.496719,10000,01000,0.200000,0.20,"[0.8644894361000001, 0.3660726845, 0.739655196...","[0.9118559361, 0.9015911818000001, 0.002224959...",0.500000,0.370370,0.587031,0.423775,0.086674,0.461202
2215,6003,t5_small_multiclaim,[Angela Lansbury contributed to Disney's Beaut...,0.332535,0.527549,0.796507,010,10,0.333333,0.50,"[0.0018639604000000001, 0.9938004613, 0.001941...","[0.9853401780000001, 0.06975777450000001]",0.166667,1.000000,0.332582,0.975697,0.387559,0.407934


In [10]:
print(metrics.root_mean_squared_error(multi_df_gold["f_fact_gold"], multi_df_gold["f_fact"]))

0.42214772455769406


## 🧠 Model Validation

In [33]:
#list all models
models = np.unique(df["model"].to_numpy())
models

array(['gpt-4-turbo-3-shot', 'qacg', 'qlora-mistral-instruct-v0.2',
       't5_small_diverse_7_beam_search', 't5_small_multiclaim'],
      dtype=object)

In [34]:
def eval_metrics(model_name):
    model_df = df[df["model"] == model_name]
    model_df_gold = gold_df[gold_df["model"] == model_name]
    model_df_multi = multi_df[multi_df["model"] == model_name]
    model_df_multi_gold = multi_df_gold[multi_df_gold["model"] == model_name]

    faithfulness = model_df["F_lg"].to_numpy() >= 0.5
    faithfulness_gold = model_df_gold["F_gold"].to_numpy()

    f_mean = np.mean(faithfulness)
    f_mean_gold = np.mean(faithfulness_gold)

    fluency_scribendi = model_df["scribendi_score_l"].to_numpy() != 1
    fluency_pplxt = model_df["pplxt_generated"].to_numpy() >= 10.37 #change
    fluency_gold = model_df_gold["G_gold"].to_numpy()

    g_mean_scribendi = np.mean(fluency_scribendi)
    g_mean_pplxt = np.mean(fluency_pplxt)
    g_mean_gold = np.mean(fluency_gold)

    decont = model_df["D"].to_numpy()
    decont_gold = model_df_gold["D_gold"].to_numpy()

    d_mean = np.mean(decont)
    d_mean_gold = np.mean(decont_gold)

    atom = model_df["A"].to_numpy()
    atom_gold = model_df_gold["A_gold"].to_numpy()

    a_mean = np.mean(atom)
    a_mean_gold = np.mean(atom_gold)

    focus_mean = np.mean(model_df_multi["Foc"].to_numpy())
    foc_mean_gold = np.mean(model_df_multi_gold["Foc_gold"].to_numpy())

    coverage_mean = np.mean(model_df_multi["Cov"].to_numpy())
    cov_mean_gold = np.mean(model_df_multi_gold["Cov_gold"].to_numpy())

    harm_foc_cov = 2*(focus_mean*coverage_mean)/(focus_mean+coverage_mean)
    harm_foc_cov_gold = 2*(foc_mean_gold*cov_mean_gold)/(foc_mean_gold+cov_mean_gold)
    

    redundancy_mean = np.mean(model_df_multi["Red"])
    return f_mean, f_mean_gold, g_mean_scribendi, g_mean_pplxt, g_mean_gold, d_mean, d_mean_gold, a_mean, a_mean_gold, focus_mean, foc_mean_gold, coverage_mean, cov_mean_gold, redundancy_mean, harm_foc_cov, harm_foc_cov_gold



In [35]:
model_val_df = pd.DataFrame(columns=["model", "F", "F_gold", "G_scribendi", "G_pplxt", "G_gold", "D", "D_gold", "A", "A_gold", "Focus", "Focus_gold", "Coverage", "Coverage_gold", "Redundancy", "F-factscore", "F-factscore_gold"])
model_val_df.set_index(keys=["model"], inplace=True)

for model in models:
    res = eval_metrics(model)

    model_val_df.loc[model] = res

In [37]:
model_val_df

,F,F_gold,G_scribendi,G_pplxt,G_gold,D,D_gold,A,A_gold,Focus,Focus_gold,Coverage,Coverage_gold,Redundancy,F-factscore,F-factscore_gold
model,,,,,,,,,,,,,,,,
gpt-4-turbo-3-shot,0.992023,0.925414,0.704859,0.987431,0.966851,0.770365,0.958564,0.917815,0.975138,0.214287,0.242012,0.811646,0.797083,0.142192,0.339057,0.371292
qacg,0.884503,0.758403,0.685363,0.991206,0.848739,0.704319,0.911765,0.888997,0.985294,0.196057,0.186340,0.671986,0.602564,0.439032,0.303551,0.284652
qlora-mistral-instruct-v0.2,0.947338,0.907895,0.747106,0.984954,0.953947,0.800926,0.960526,0.949074,1.000000,0.580968,0.595913,0.633668,0.686667,0.191211,0.606175,0.638079
t5_small_diverse_7_beam_search,0.916571,0.794393,0.738178,0.991157,0.887850,0.803153,0.953271,0.951942,0.985981,0.554161,0.508690,0.575715,0.617500,0.593566,0.564732,0.557839
t5_small_multiclaim,0.946889,0.883041,0.758725,0.993424,0.912281,0.816894,0.941520,0.959535,1.000000,0.581719,0.470952,0.514516,0.500000,0.541528,0.546057,0.485042


In [115]:
model_val_df.to_latex()

/tmp/ipykernel_1295493/1207153265.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  model_val_df.to_latex()


'\\begin{tabular}{lrrrrrrrrrrrrrrrr}\n\\toprule\n{} &         F &    F\\_gold &  G\\_scribendi &   G\\_pplxt &    G\\_gold &         D &    D\\_gold &         A &    A\\_gold &     Focus &  Focus\\_gold &  Coverage &  Coverage\\_gold &  Redundancy &  F-factscore &  F-factscore\\_gold \\\\\nmodel                          &           &           &              &           &           &           &           &           &           &           &             &           &                &             &              &                   \\\\\n\\midrule\ngpt-4-turbo-3-shot             &  0.992023 &  0.925414 &     0.704859 &  0.985497 &  0.966851 &  0.770365 &  0.958564 &  0.917815 &  0.975138 &  0.214287 &    0.242012 &  0.811646 &       0.797083 &    0.142192 &     0.339057 &          0.371292 \\\\\nqacg                           &  0.884503 &  0.758403 &     0.685363 &  0.989642 &  0.848739 &  0.704319 &  0.911765 &  0.888997 &  0.985294 &  0.196057 &    0.186340 &  0.671986 &       0.6025

# 🧍🧍🧍🧍🧍 Inter-Anotator Agreement

## Single

### Load and Preprocess Data

In [30]:
inter_df = pd.read_json("/mnt/data/factcheck/_acl24/cross_annotations.json", dtype=str)

In [31]:
inter_df

,1859: Crown colony was a type of colonial administration of the overseas territories of the British Empire.,1859: A Crown colony has been known as a British Overseas Territory since 2002.,1859: Another name for a Crown colony is British.,1859: A crown colony was known as a royal colony in the 17th century.,"2342: Along with TBS, MBS broadcasts Code Geass.",2342: The sequel to Code Geass aired on the television station JNN.,2342: Bandai Entertainment licensed most of the Code Geass series for English release in 2007.,2342: Bandai Entertainment aired two TV series on Adult Swim in 2007.,2342: The English version of Code Geass was released in December 2007.,2342: Bandai Entertainment aired the two episodes of Code Geass on Adult Swim.,...,2571: Ecuador has an ethnically diverse population.,2571: Ecuador had some 15.2 million people.,2571: Ecuador has ethnically diverse populations.,1298: Richard Ramirez's home invasion crime spree terrorized the residents of the greater Los Angeles area.,1298: Richard Ramirez threw out a home invasion.,1298: Richard Ramirez had an home invasion spree.,6003: Angela Lansbury has worked with Walt Disney.,6003: Angela Lansbury was a voice actress.,6003: Angela Lansbury worked on Beauty and the Beast.,6003: Angela Lansbury is an actress.
356,3311,2211,1111,3311,3311,3311,3311,2311,3311,1311,...,3311,2301,3211,3311,1111,3211,1311,3311,3311,3311
360,3311,3311,1301,3311,1311,3311,3311,3301,1311,1301,...,3311,3101,3211,nan,nan,nan,nan,nan,nan,nan
353,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,2311,2311,3311,3311
358,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
354,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,3311,1011,3211,nan,nan,nan,nan


In [4]:
def app_funf(x):
    if x != 'nan':
        return [int(digit) for digit in x][0]
    else:
        return x
    
def app_fung(x):
    if x != 'nan':
        return [int(digit) for digit in x][1]
    else:
        return x
    

def app_fund(x):
    if x != 'nan':
        return [int(digit) for digit in x][2]
    else:
        return x
    

def app_funa(x):
    if x != 'nan':
        return [int(digit) for digit in x][3]
    else:
        return x

In [5]:
df_f = inter_df.applymap(app_funf)
df_g = inter_df.applymap(app_fung)
df_d = inter_df.applymap(app_fund)
df_a = inter_df.applymap(app_funa)

In [6]:
df_f

,1859: Crown colony was a type of colonial administration of the overseas territories of the British Empire.,1859: A Crown colony has been known as a British Overseas Territory since 2002.,1859: Another name for a Crown colony is British.,1859: A crown colony was known as a royal colony in the 17th century.,"2342: Along with TBS, MBS broadcasts Code Geass.",2342: The sequel to Code Geass aired on the television station JNN.,2342: Bandai Entertainment licensed most of the Code Geass series for English release in 2007.,2342: Bandai Entertainment aired two TV series on Adult Swim in 2007.,2342: The English version of Code Geass was released in December 2007.,2342: Bandai Entertainment aired the two episodes of Code Geass on Adult Swim.,...,2571: Ecuador has an ethnically diverse population.,2571: Ecuador had some 15.2 million people.,2571: Ecuador has ethnically diverse populations.,1298: Richard Ramirez's home invasion crime spree terrorized the residents of the greater Los Angeles area.,1298: Richard Ramirez threw out a home invasion.,1298: Richard Ramirez had an home invasion spree.,6003: Angela Lansbury has worked with Walt Disney.,6003: Angela Lansbury was a voice actress.,6003: Angela Lansbury worked on Beauty and the Beast.,6003: Angela Lansbury is an actress.
356,3,2,1,3,3,3,3,2,3,1,...,3,2,3,3,1,3,1,3,3,3
360,3,3,1,3,1,3,3,3,1,1,...,3,3,3,nan,nan,nan,nan,nan,nan,nan
353,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,2,2,3,3
358,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
354,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,3,1,3,nan,nan,nan,nan


In [7]:
f_np = df_f.to_numpy(dtype=np.float64)
g_np = df_g.to_numpy(dtype=np.float64)
d_np = df_d.to_numpy(dtype=np.float64)
a_np = df_a.to_numpy(dtype=np.float64)

### Compute Agreements

Krippendorf alphas using krippendorf library (enables different level of measurements -> better consider them as ordinal)

In [25]:
import krippendorff

#faithfullness (F)
display(krippendorff.alpha(reliability_data=f_np, level_of_measurement="ordinal", value_domain=[3,2,1,0]))
display(krippendorff.alpha(reliability_data=f_np, level_of_measurement="nominal", value_domain=[3,2,1,0]))

#fluency (G)
display(krippendorff.alpha(reliability_data=g_np, level_of_measurement="ordinal", value_domain=[3,2,1,0]))
display(krippendorff.alpha(reliability_data=g_np, level_of_measurement="nominal", value_domain=[3,2,1,0]))

#decont (D)
display(krippendorff.alpha(reliability_data=d_np, level_of_measurement="ordinal", value_domain=[1,0]))
display(krippendorff.alpha(reliability_data=d_np, level_of_measurement="nominal", value_domain=[1,0]))

#atom (A)
display(krippendorff.alpha(reliability_data=a_np, level_of_measurement="ordinal", value_domain=[1,0]))
display(krippendorff.alpha(reliability_data=a_np, level_of_measurement="nominal", value_domain=[1,0]))

0.5263607019198895

0.4590888754375345

0.412255866050142

0.3305936414241417

0.27234309623430963

0.27234309623430963

-0.013461538461538414

-0.013461538461538414

irrCAC library (made by Gwet for R, unofficial Python port), computes only nominal krippendorf but also gwet's AC1 and fleiss's kappa

In [90]:
from irrCAC.raw import CAC

cac_f = CAC(pd.DataFrame(f_np.T), categories=[3,2,1,0])
display(cac_f)
display(cac_f.gwet())
display(cac_f.krippendorff()) #krippendorf alpha only for nominal values

<irrCAC.raw.CAC Subjects: 251, Raters: 5, Categories: [3, 2, 1, 0], Weights: "identity">

{'est': {'coefficient_value': 0.83457,
  'coefficient_name': 'AC1',
  'confidence_interval': (0.78424, 0.8849),
  'p_value': 0.0,
  'z': 32.65737,
  'se': 0.02556,
  'pa': 0.84993,
  'pe': 0.09287},
 'weights': array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 'categories': [3, 2, 1, 0]}

{'est': {'coefficient_value': 0.45909,
  'coefficient_name': "Krippendorff's Alpha",
  'confidence_interval': (0.32423, 0.59395),
  'p_value': 1.3351053596011297e-10,
  'z': 6.70464,
  'se': 0.06847,
  'pa': 0.85255,
  'pe': 0.72741},
 'weights': array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 'categories': [3, 2, 1, 0]}

In [91]:
from irrCAC.raw import CAC

cac_f = CAC(pd.DataFrame(f_np.T), categories=[0,1,2,3], weights="ordinal")
display(cac_f)
display(cac_f.gwet())
display(cac_f.krippendorff()) #krippendorf alpha only for nominal values

<irrCAC.raw.CAC Subjects: 251, Raters: 5, Categories: [0, 1, 2, 3], Weights: "ordinal">

{'est': {'coefficient_value': 0.93728,
  'coefficient_name': 'AC2',
  'confidence_interval': (0.9139, 0.96066),
  'p_value': 0.0,
  'z': 78.94277,
  'se': 0.01187,
  'pa': 0.9533,
  'pe': 0.25538},
 'weights': array([[1.        , 0.83333333, 0.5       , 0.        ],
        [0.83333333, 1.        , 0.83333333, 0.5       ],
        [0.5       , 0.83333333, 1.        , 0.83333333],
        [0.        , 0.5       , 0.83333333, 1.        ]]),
 'categories': [0, 1, 2, 3]}

{'est': {'coefficient_value': 0.56697,
  'coefficient_name': "Krippendorff's Alpha",
  'confidence_interval': (0.42477, 0.70916),
  'p_value': 1.199040866595169e-13,
  'z': 7.8529,
  'se': 0.0722,
  'pa': 0.954,
  'pe': 0.89378},
 'weights': array([[1.        , 0.83333333, 0.5       , 0.        ],
        [0.83333333, 1.        , 0.83333333, 0.5       ],
        [0.5       , 0.83333333, 1.        , 0.83333333],
        [0.        , 0.5       , 0.83333333, 1.        ]]),
 'categories': [0, 1, 2, 3]}

In [92]:
cac_g = CAC(pd.DataFrame(g_np.T))
display(cac_g)
display(cac_g.gwet())
display(cac_g.krippendorff())

<irrCAC.raw.CAC Subjects: 251, Raters: 5, Categories: [0.0, 1.0, 2.0, 3.0], Weights: "identity">

{'est': {'coefficient_value': 0.86072,
  'coefficient_name': 'AC1',
  'confidence_interval': (0.81467, 0.90678),
  'p_value': 0.0,
  'z': 36.80788,
  'se': 0.02338,
  'pa': 0.86985,
  'pe': 0.06557},
 'weights': array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 'categories': [0.0, 1.0, 2.0, 3.0]}

{'est': {'coefficient_value': 0.33059,
  'coefficient_name': "Krippendorff's Alpha",
  'confidence_interval': (0.17658, 0.48461),
  'p_value': 3.315528155378722e-05,
  'z': 4.22748,
  'se': 0.0782,
  'pa': 0.87146,
  'pe': 0.80797},
 'weights': array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 'categories': [0.0, 1.0, 2.0, 3.0]}

In [93]:
cac_g = CAC(pd.DataFrame(g_np.T), weights="ordinal")
display(cac_g)
display(cac_g.gwet())
display(cac_g.krippendorff())

<irrCAC.raw.CAC Subjects: 251, Raters: 5, Categories: [0.0, 1.0, 2.0, 3.0], Weights: "ordinal">

{'est': {'coefficient_value': 0.95329,
  'coefficient_name': 'AC2',
  'confidence_interval': (0.93317, 0.9734),
  'p_value': 0.0,
  'z': 93.33375,
  'se': 0.01021,
  'pa': 0.96171,
  'pe': 0.18031},
 'weights': array([[1.        , 0.83333333, 0.5       , 0.        ],
        [0.83333333, 1.        , 0.83333333, 0.5       ],
        [0.5       , 0.83333333, 1.        , 0.83333333],
        [0.        , 0.5       , 0.83333333, 1.        ]]),
 'categories': [0.0, 1.0, 2.0, 3.0]}

{'est': {'coefficient_value': 0.37321,
  'coefficient_name': "Krippendorff's Alpha",
  'confidence_interval': (0.18804, 0.55839),
  'p_value': 9.421123692421318e-05,
  'z': 3.96938,
  'se': 0.09402,
  'pa': 0.96282,
  'pe': 0.94069},
 'weights': array([[1.        , 0.83333333, 0.5       , 0.        ],
        [0.83333333, 1.        , 0.83333333, 0.5       ],
        [0.5       , 0.83333333, 1.        , 0.83333333],
        [0.        , 0.5       , 0.83333333, 1.        ]]),
 'categories': [0.0, 1.0, 2.0, 3.0]}

In [94]:
cac_d = CAC(pd.DataFrame(d_np.T))
display(cac_d)
display(cac_d.gwet())
display(cac_d.krippendorff())

<irrCAC.raw.CAC Subjects: 251, Raters: 5, Categories: [0.0, 1.0], Weights: "identity">

{'est': {'coefficient_value': 0.85163,
  'coefficient_name': 'AC1',
  'confidence_interval': (0.79744, 0.90583),
  'p_value': 0.0,
  'z': 30.94842,
  'se': 0.02752,
  'pa': 0.87649,
  'pe': 0.16756},
 'weights': array([[1., 0.],
        [0., 1.]]),
 'categories': [0.0, 1.0]}

{'est': {'coefficient_value': 0.27234,
  'coefficient_name': "Krippendorff's Alpha",
  'confidence_interval': (0.11397, 0.43072),
  'p_value': 0.0008212665778950523,
  'z': 3.38673,
  'se': 0.08041,
  'pa': 0.87524,
  'pe': 0.82854},
 'weights': array([[1., 0.],
        [0., 1.]]),
 'categories': [0.0, 1.0]}

In [95]:
cac_a = CAC(pd.DataFrame(a_np.T))
display(cac_a)
display(cac_a.gwet())
display(cac_a.krippendorff())

<irrCAC.raw.CAC Subjects: 251, Raters: 5, Categories: [0.0, 1.0], Weights: "identity">

{'est': {'coefficient_value': 0.96851,
  'coefficient_name': 'AC1',
  'confidence_interval': (0.94608, 0.99094),
  'p_value': 0.0,
  'z': 85.03754,
  'se': 0.01139,
  'pa': 0.96946,
  'pe': 0.03008},
 'weights': array([[1., 0.],
        [0., 1.]]),
 'categories': [0.0, 1.0]}

{'est': {'coefficient_value': -0.01346,
  'coefficient_name': "Krippendorff's Alpha",
  'confidence_interval': (-0.02419, -0.00274),
  'p_value': 0.014090934605453409,
  'z': -2.47202,
  'se': 0.00544,
  'pa': 0.96975,
  'pe': 0.97016},
 'weights': array([[1., 0.],
        [0., 1.]]),
 'categories': [0.0, 1.0]}

In [9]:
def percentage_agreement(data: np.array):
    """
    Compute percentages agreement. Returns agreement between pair of anotators and agreement between annotators as a whole.
    """
    pairs = []
    orig_eq = []

    for i in range(data.shape[0]):
        #get one row (one annotated sample)
        x = np.array(data[i, :][~np.isnan(data[i, :])], dtype=np.int8)

        #make combinations of the row (each anotator against every else)
        combs = list(combinations(x, 2))
        #append combinations to list for further coputation
        for comb in combs:
            pairs.append(list(comb))

        #append also original annotations for a full-annotation match
        original_row = list(x)
        orig_eq.append(original_row.count(original_row[0]) == len(original_row))

    #convert pairs to numpy array
    pairs = np.array(pairs, dtype=int)

    return metrics.accuracy_score(pairs[:,0], pairs[:,1]), np.sum(orig_eq) / len(orig_eq)

In [10]:
display(percentage_agreement(f_np.T))
display(percentage_agreement(g_np.T))
display(percentage_agreement(d_np.T))
display(percentage_agreement(a_np.T))

(0.858085808580858, 0.8446215139442231)

(0.8745874587458746, 0.8645418326693227)

(0.8712871287128713, 0.8685258964143426)

(0.9702970297029703, 0.9681274900398407)

try only Cohen kappa on 2 annotators

In [47]:
pd.DataFrame(f_np).isna().sum(axis=1)

0     23
1    165
2    168
3    225
4    146
dtype: int64

In [57]:
two_annotators = pd.DataFrame(f_np[:2,:]).dropna(axis=1)
metrics.cohen_kappa_score(two_annotators.iloc[0].to_numpy(), two_annotators.iloc[1].to_numpy(), labels=[0,1,2,3])

0.4996121024049651

In [59]:
two_annotators_0_4 = pd.DataFrame(f_np[[0,4],:]).dropna(axis=1)
metrics.cohen_kappa_score(two_annotators_0_4.iloc[0].to_numpy(), two_annotators_0_4.iloc[1].to_numpy(), labels=[0,1,2,3])

0.45489078822412155

In [61]:
two_annotators = pd.DataFrame(g_np[:2,:]).dropna(axis=1)
metrics.cohen_kappa_score(two_annotators.iloc[0].to_numpy(), two_annotators.iloc[1].to_numpy(), labels=[0,1,2,3])

0.21763470958712394

In [62]:
two_annotators_0_4 = pd.DataFrame(g_np[[0,4],:]).dropna(axis=1)
metrics.cohen_kappa_score(two_annotators_0_4.iloc[0].to_numpy(), two_annotators_0_4.iloc[1].to_numpy(), labels=[0,1,2,3])

0.37733644859813087

In [69]:
pd.DataFrame(a_np[:2,:]).dropna(axis=1)

,0,1,2,3,4,5,6,7,8,9,...,234,235,236,237,238,239,240,241,242,243
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [76]:
two_annotators = pd.DataFrame(a_np[:2,:]).dropna(axis=1)
display(two_annotators.iloc[0].to_numpy())
display(two_annotators.iloc[1].to_numpy())
metrics.cohen_kappa_score(two_annotators.iloc[0].to_numpy(), two_annotators.iloc[1].to_numpy(), labels=[0,1])

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1.])

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1.])

0.0

In [77]:
two_annotators = pd.DataFrame(d_np[:2,:]).dropna(axis=1)
display(two_annotators.iloc[0].to_numpy())
display(two_annotators.iloc[1].to_numpy())
metrics.cohen_kappa_score(two_annotators.iloc[0].to_numpy(), two_annotators.iloc[1].to_numpy(), labels=[0,1])

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       1.])

array([1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
       1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       1.])

0.09314586994727592

In [84]:
two_annotators_gwet = pd.DataFrame(d_np[:2,:]).dropna(axis=1)
two_annotators_gwet

,0,1,2,3,4,5,6,7,8,9,...,234,235,236,237,238,239,240,241,242,243
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
1,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0


In [86]:
cac_d_bin = CAC(two_annotators_gwet)
display(cac_d_bin)
display(cac_d_bin.gwet())
display(cac_d_bin.krippendorff())

<irrCAC.raw.CAC Subjects: 2, Raters: 86, Categories: [0.0, 1.0], Weights: "identity">

{'est': {'coefficient_value': 0.82724,
  'coefficient_name': 'AC1',
  'confidence_interval': (-0.55101, 1),
  'p_value': 0.08300234285636576,
  'z': 7.62639,
  'se': 0.10847,
  'pa': 0.85308,
  'pe': 0.14954},
 'weights': array([[1., 0.],
        [0., 1.]]),
 'categories': [0.0, 1.0]}

{'est': {'coefficient_value': 0.02322,
  'coefficient_name': "Krippendorff's Alpha",
  'confidence_interval': (-0.17049, 0.21693),
  'p_value': 0.3698505832477834,
  'z': 1.52307,
  'se': 0.01525,
  'pa': 0.85393,
  'pe': 0.85046},
 'weights': array([[1., 0.],
        [0., 1.]]),
 'categories': [0.0, 1.0]}

Prevalence

In [11]:
np.unique(f_np, return_counts=True)

(array([ 1.,  2.,  3., nan]), array([ 57,  25, 446, 727]))

In [12]:
np.unique(g_np, return_counts=True)

(array([ 0.,  1.,  2.,  3., nan]), array([  2,  19,  34, 473, 727]))

In [13]:
np.unique(a_np, return_counts=True)

(array([ 0.,  1., nan]), array([  8, 520, 727]))

In [14]:
np.unique(d_np, return_counts=True)

(array([ 0.,  1., nan]), array([ 50, 478, 727]))

## Multi

In [71]:
def percentage_agreement(data: np.array):
    """
    Compute percentages agreement. Returns agreement between pair of anotators and agreement between annotators as a whole.
    """
    pairs = []
    orig_eq = []

    for i in range(data.shape[0]):
        #get one row (one annotated sample)
        x = np.array(data[i, :][~np.isnan(data[i, :])], dtype=np.int8)

        #make combinations of the row (each anotator against every else)
        combs = list(combinations(x, 2))
        #append combinations to list for further coputation
        for comb in combs:
            pairs.append(list(comb))

        #append also original annotations for a full-annotation match
        original_row = list(x)
        orig_eq.append(original_row.count(original_row[0]) == len(original_row))

    #convert pairs to numpy array
    pairs = np.array(pairs, dtype=int)

    return metrics.accuracy_score(pairs[:,0], pairs[:,1]), np.sum(orig_eq) / len(orig_eq)

### Load and Preprocess Data

In [49]:
import json

with open("/mnt/data/factcheck/_acl24/cross_multiannotations.json", "r") as f:
    multi_annotations = json.load(f)

focus_flattened = {}
focus_aggregated = {}
coverage_flattened = {}
coverage_aggregated = {}


for data_point_key, data_point in multi_annotations.items():
    focus_annots = {}
    coverage_annots = {}
    for annotator_id, annotations in data_point.items():
        focuses = [int(foc) for foc in annotations["Foc"]]
        coverages = [int(cov) for cov in annotations["Cov"]]
        for i, focus in enumerate(focuses):
            key = f"{data_point_key}_{i}"
            if key in focus_flattened:
                focus_flattened[key][annotator_id] = int(focus)
            else:
                focus_flattened[key] = {annotator_id: int(focus)}

        if data_point_key in focus_aggregated:
            focus_aggregated[data_point_key][annotator_id] = np.average(focuses)
        else:
            focus_aggregated[data_point_key] = {annotator_id: np.average(focuses)}

        for i, coverage in enumerate(coverages):
            key = f"{data_point_key}_{i}"
            if key in coverage_flattened:
                coverage_flattened[key][annotator_id] = int(coverage)
            else:
                coverage_flattened[key] = {annotator_id: int(coverage)}

        if data_point_key in coverage_aggregated:
            coverage_aggregated[data_point_key][annotator_id] = np.average(coverages)
        else:
            coverage_aggregated[data_point_key] = {annotator_id: np.average(coverages)}

print(focus_flattened)
print(coverage_flattened)
print(focus_aggregated)
print(coverage_aggregated)


{'1859: qacg_0': {'356': 1, '360': 0}, '1859: qacg_1': {'356': 1, '360': 0}, '1859: qacg_2': {'356': 0, '360': 0}, '1859: qacg_3': {'356': 0, '360': 0}, '2342: qacg_0': {'360': 0, '356': 0}, '2342: qacg_1': {'360': 0, '356': 0}, '2342: qacg_2': {'360': 0, '356': 0}, '2342: qacg_3': {'360': 0, '356': 0}, '2342: qacg_4': {'360': 0, '356': 0}, '2342: qacg_5': {'360': 0, '356': 0}, '2342: qacg_6': {'360': 0, '356': 0}, '2342: qacg_7': {'360': 0, '356': 0}, '2342: qacg_8': {'360': 0, '356': 0}, '6003: qacg_0': {'353': 0, '356': 0}, '6003: qacg_1': {'353': 0, '356': 0}, '6003: qacg_2': {'353': 0, '356': 0}, '6003: qacg_3': {'353': 0, '356': 0}, '6003: qacg_4': {'353': 0, '356': 0}, '6003: qacg_5': {'353': 0, '356': 0}, '6003: qacg_6': {'356': 0}, '2029: qacg_0': {'358': 0, '353': 0, '356': 0}, '2029: qacg_1': {'358': 0, '353': 0, '356': 0}, '2029: qacg_2': {'358': 1, '353': 1, '356': 1}, '2029: qacg_3': {'358': 0, '353': 0, '356': 0}, '2029: qacg_4': {'358': 0, '353': 0, '356': 0}, '2029: qa

### Process flattened Focus

In [51]:
focus_flattened_df = pd.DataFrame(focus_flattened)
focus_flattened_df

,1859: qacg_0,1859: qacg_1,1859: qacg_2,1859: qacg_3,2342: qacg_0,2342: qacg_1,2342: qacg_2,2342: qacg_3,2342: qacg_4,2342: qacg_5,...,1298: t5_small_diverse_7_beam_search_2,1298: t5_small_diverse_7_beam_search_3,1298: t5_small_diverse_7_beam_search_4,1298: t5_small_diverse_7_beam_search_5,6003: t5_small_diverse_7_beam_search_0,6003: t5_small_diverse_7_beam_search_1,6003: t5_small_diverse_7_beam_search_2,6003: t5_small_diverse_7_beam_search_3,6003: t5_small_diverse_7_beam_search_4,6003: t5_small_diverse_7_beam_search_5
356,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0
360,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0
358,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
#convert to numpy
focus_flattened_np = focus_flattened_df.to_numpy(dtype=np.float64)
focus_flattened_np

array([[ 1.,  1.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ..., nan, nan, nan],
       [nan, nan, nan, ...,  0.,  0.,  0.],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [61]:
#krippendorff alpha
import krippendorff

display(krippendorff.alpha(reliability_data=focus_flattened_np, level_of_measurement="ordinal"))

0.7547042477173481

In [62]:
from irrCAC.raw import CAC
cac_a = CAC(pd.DataFrame(focus_flattened_np.T))
display(cac_a)
display(cac_a.gwet())
display(cac_a.krippendorff())

<irrCAC.raw.CAC Subjects: 317, Raters: 5, Categories: [0.0, 1.0], Weights: "identity">

{'est': {'coefficient_value': 0.79552,
  'coefficient_name': 'AC1',
  'confidence_interval': (0.73035, 0.86069),
  'p_value': 0.0,
  'z': 24.01717,
  'se': 0.03312,
  'pa': 0.88819,
  'pe': 0.45318},
 'weights': array([[1., 0.],
        [0., 1.]]),
 'categories': [0.0, 1.0]}

{'est': {'coefficient_value': 0.7547,
  'coefficient_name': "Krippendorff's Alpha",
  'confidence_interval': (0.68001, 0.8294),
  'p_value': 0.0,
  'z': 19.88049,
  'se': 0.03796,
  'pa': 0.88955,
  'pe': 0.54974},
 'weights': array([[1., 0.],
        [0., 1.]]),
 'categories': [0.0, 1.0]}

In [72]:
display(percentage_agreement(focus_flattened_np.T))

(0.8923076923076924, 0.8832807570977917)

### Process flattened Coverage

In [78]:
coverage_flattened_df = pd.DataFrame(coverage_flattened)
coverage_flattened_df

,1859: qacg_0,1859: qacg_1,1859: qacg_2,1859: qacg_3,1859: qacg_4,2342: qacg_0,2342: qacg_1,2342: qacg_2,2342: qacg_3,2342: qacg_4,...,2571: t5_small_diverse_7_beam_search_2,2571: t5_small_diverse_7_beam_search_3,2571: t5_small_diverse_7_beam_search_4,1298: t5_small_diverse_7_beam_search_0,1298: t5_small_diverse_7_beam_search_1,1298: t5_small_diverse_7_beam_search_2,1298: t5_small_diverse_7_beam_search_3,1298: t5_small_diverse_7_beam_search_4,6003: t5_small_diverse_7_beam_search_0,6003: t5_small_diverse_7_beam_search_1
356,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0
360,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0
358,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,1.0,0.0,0.0,1.0,NaN,NaN


In [80]:
coverage_flattened_np = coverage_flattened_df.to_numpy(dtype=np.float64)
coverage_flattened_np

array([[ 1.,  0.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,
         1.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  0.,
         1.,  0.,  1.,  1.,  1., nan, nan, nan,  1.,  1.,  1.,  0.,  0.,
         0.,  1.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  0.,  1.,  1.,  1.,  1., nan, nan, nan,  1.,  1.,
         1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,
         0.,  1.,  0.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,
         1.,  1.,  0.,  1.,  0.,  1.,  1.,  0., nan, nan, nan,  1.,  1.,
         0.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  0.,
         0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,
         1.,  1.,  0.,  0.,  0.,  1.,  1.,  0., nan, nan, nan,  1.,  0.,
         0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,
         0.,  1.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  1.,  1.,
         0.,  1.,  1.,  0.,  1.,  1.,  1.,  0., nan

In [81]:
#krippendorff alpha
import krippendorff

display(krippendorff.alpha(reliability_data=coverage_flattened_np, level_of_measurement="ordinal"))

0.6384926113412077

In [91]:
from irrCAC.raw import CAC
cac_a = CAC(pd.DataFrame(coverage_flattened_np.T))
display(cac_a)
display(cac_a.gwet())
display(cac_a.krippendorff())

<irrCAC.raw.CAC Subjects: 195, Raters: 5, Categories: [0.0, 1.0], Weights: "identity">

{'est': {'coefficient_value': 0.66208,
  'coefficient_name': 'AC1',
  'confidence_interval': (0.55629, 0.76788),
  'p_value': 0.0,
  'z': 12.34288,
  'se': 0.05364,
  'pa': 0.82393,
  'pe': 0.47896},
 'weights': array([[1., 0.],
        [0., 1.]]),
 'categories': [0.0, 1.0]}

{'est': {'coefficient_value': 0.63849,
  'coefficient_name': "Krippendorff's Alpha",
  'confidence_interval': (0.52888, 0.74811),
  'p_value': 0.0,
  'z': 11.48803,
  'se': 0.05558,
  'pa': 0.82759,
  'pe': 0.52307},
 'weights': array([[1., 0.],
        [0., 1.]]),
 'categories': [0.0, 1.0]}

In [93]:
display(percentage_agreement(coverage_flattened_np.T))

(0.8355555555555556, 0.8205128205128205)

### Process Aggregated Focus

In [84]:
focus_aggregated_df = pd.DataFrame(focus_aggregated)
focus_aggregated_df

,1859: qacg,2342: qacg,6003: qacg,2029: qacg,1298: qacg,785: qacg,3809: qacg,606: qacg,1991: qacg,2571: qacg,...,3809: t5_small_diverse_7_beam_search,785: t5_small_diverse_7_beam_search,1859: t5_small_diverse_7_beam_search,606: t5_small_diverse_7_beam_search,2342: t5_small_diverse_7_beam_search,1991: t5_small_diverse_7_beam_search,2029: t5_small_diverse_7_beam_search,2571: t5_small_diverse_7_beam_search,1298: t5_small_diverse_7_beam_search,6003: t5_small_diverse_7_beam_search
356,0.5,0.0,0.0,0.133333,0.181818,0.105263,0.500000,0.3,NaN,0.071429,...,0.8,0.50,0.75,1.0,0.8,NaN,0.75,0.428571,0.666667,0.500000
360,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.071429,...,NaN,NaN,0.50,NaN,1.0,NaN,NaN,0.571429,NaN,NaN
353,NaN,NaN,0.0,0.133333,NaN,NaN,0.333333,NaN,0.181818,NaN,...,0.6,NaN,NaN,NaN,NaN,1.0,0.25,NaN,NaN,0.166667
358,NaN,NaN,NaN,0.133333,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.50,NaN,NaN,NaN
354,NaN,NaN,NaN,NaN,0.090909,0.052632,NaN,0.3,0.181818,NaN,...,NaN,0.25,NaN,0.8,NaN,1.0,NaN,NaN,0.166667,NaN


In [85]:
focus_aggregated_np = focus_aggregated_df.to_numpy(dtype=np.float64)
focus_aggregated_np

array([[0.5       , 0.        , 0.        , 0.13333333, 0.18181818,
        0.10526316, 0.5       , 0.3       ,        nan, 0.07142857,
        0.66666667, 0.07692308, 0.4       , 0.42857143, 0.3       ,
               nan, 0.18181818, 0.2       , 0.25      , 0.16666667,
        1.        , 0.        , 0.        , 0.75      , 1.        ,
               nan, 0.5       , 1.        , 0.66666667, 0.33333333,
        1.        , 0.        , 0.66666667, 0.66666667, 1.        ,
               nan, 1.        , 0.        , 0.2       , 0.33333333,
        0.8       , 0.5       , 0.75      , 1.        , 0.8       ,
               nan, 0.75      , 0.42857143, 0.66666667, 0.5       ],
       [0.        , 0.        ,        nan,        nan,        nan,
               nan,        nan,        nan,        nan, 0.07142857,
               nan,        nan, 0.        ,        nan, 0.3       ,
               nan,        nan, 0.13333333,        nan,        nan,
               nan,        nan, 0.8       ,    

In [89]:
#krippendorff alpha
import krippendorff

display(krippendorff.alpha(reliability_data=focus_aggregated_np, level_of_measurement="ratio"))

0.6985739493362138

In [111]:
from irrCAC.raw import CAC
cac_a = CAC(pd.DataFrame(focus_aggregated_np.T))
display(cac_a)
display(cac_a.gwet())
display(cac_a.krippendorff())

<irrCAC.raw.CAC Subjects: 50, Raters: 5, Categories: [0.0, 0.05263157894736842, 0.07142857142857142, 0.07692307692307693, 0.09090909090909091, 0.10526315789473684, 0.13333333333333333, 0.16666666666666666, 0.18181818181818182, 0.2, 0.2222222222222222, 0.25, 0.3, 0.3333333333333333, 0.375, 0.4, 0.42857142857142855, 0.5, 0.5714285714285714, 0.6, 0.6666666666666666, 0.75, 0.8, 0.8333333333333334, 1.0], Weights: "identity">

{'est': {'coefficient_value': 0.5354,
  'coefficient_name': 'AC1',
  'confidence_interval': (0.38953, 0.68127),
  'p_value': 1.7383381401003817e-09,
  'z': 7.37605,
  'se': 0.07259,
  'pa': 0.55333,
  'pe': 0.03859},
 'weights': array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0

{'est': {'coefficient_value': 0.522,
  'coefficient_name': "Krippendorff's Alpha",
  'confidence_interval': (0.37382, 0.67018),
  'p_value': 5.000051928050198e-09,
  'z': 7.07926,
  'se': 0.07374,
  'pa': 0.55664,
  'pe': 0.07247},
 'weights': array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0.

In [94]:
display(percentage_agreement(focus_aggregated_np.T))

(0.9, 0.9)

### Process Aggregated Coverage

In [95]:
coverage_aggregated_df = pd.DataFrame(coverage_aggregated)
coverage_aggregated_df

,1859: qacg,2342: qacg,6003: qacg,2029: qacg,1298: qacg,785: qacg,3809: qacg,606: qacg,1991: qacg,2571: qacg,...,3809: t5_small_diverse_7_beam_search,785: t5_small_diverse_7_beam_search,1859: t5_small_diverse_7_beam_search,606: t5_small_diverse_7_beam_search,2342: t5_small_diverse_7_beam_search,1991: t5_small_diverse_7_beam_search,2029: t5_small_diverse_7_beam_search,2571: t5_small_diverse_7_beam_search,1298: t5_small_diverse_7_beam_search,6003: t5_small_diverse_7_beam_search
356,0.8,0.333333,0.5,1.0,0.2,0.5,0.4,1.0,NaN,0.6,...,0.4,0.5,0.4,0.666667,0.666667,NaN,0.666667,0.2,0.6,1.0
360,0.8,0.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2,...,NaN,NaN,0.6,NaN,0.833333,NaN,NaN,0.6,NaN,NaN
353,NaN,NaN,0.0,1.0,NaN,NaN,0.2,NaN,1.0,NaN,...,0.2,NaN,NaN,NaN,NaN,1.0,0.666667,NaN,NaN,0.5
358,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.666667,NaN,NaN,NaN
354,NaN,NaN,NaN,NaN,0.6,1.0,NaN,1.0,1.0,NaN,...,NaN,1.0,NaN,0.666667,NaN,1.0,NaN,NaN,0.6,NaN


In [99]:
coverage_aggregated_np = coverage_aggregated_df.to_numpy(dtype=np.float64)
coverage_aggregated_np

array([[0.8       , 0.33333333, 0.5       , 1.        , 0.2       ,
        0.5       , 0.4       , 1.        ,        nan, 0.6       ,
        0.4       , 0.5       , 0.8       , 1.        , 0.83333333,
               nan, 1.        , 0.8       , 0.8       , 0.5       ,
        0.4       , 0.        , 0.8       , 1.        , 0.5       ,
               nan, 0.66666667, 0.6       , 0.8       , 0.5       ,
        0.4       , 0.        , 0.4       , 0.66666667, 0.33333333,
               nan, 0.33333333, 0.4       , 0.2       , 0.5       ,
        0.4       , 0.5       , 0.4       , 0.66666667, 0.66666667,
               nan, 0.66666667, 0.2       , 0.6       , 1.        ],
       [0.8       , 0.33333333,        nan,        nan,        nan,
               nan,        nan,        nan,        nan, 0.2       ,
               nan,        nan, 0.8       ,        nan, 1.        ,
               nan,        nan, 0.6       ,        nan,        nan,
               nan,        nan, 0.6       ,    

In [108]:
#krippendorff alpha
import krippendorff

display(krippendorff.alpha(reliability_data=coverage_aggregated_np, level_of_measurement="interval"))

0.5931757323618311

In [110]:
from irrCAC.raw import CAC
cac_a = CAC(pd.DataFrame(coverage_aggregated_np.T))
display(cac_a)
display(cac_a.gwet())
display(cac_a.krippendorff())

<irrCAC.raw.CAC Subjects: 50, Raters: 5, Categories: [0.0, 0.2, 0.3333333333333333, 0.4, 0.5, 0.6, 0.6666666666666666, 0.8, 0.8333333333333334, 1.0], Weights: "identity">

{'est': {'coefficient_value': 0.52786,
  'coefficient_name': 'AC1',
  'confidence_interval': (0.37321, 0.6825),
  'p_value': 1.0955750751051596e-08,
  'z': 6.85936,
  'se': 0.07695,
  'pa': 0.57333,
  'pe': 0.09632},
 'weights': array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]),
 'categories': [0.0,
  0.2,
  0.3333333333333333,
  0.4,
  0.5,
  0.6,
  0.6666666666666666,
  0.8,
  0.8333333333333334,
  1.0]}

{'est': {'coefficient_value': 0.51983,
  'coefficient_name': "Krippendorff's Alpha",
  'confidence_interval': (0.36536, 0.67431),
  'p_value': 1.5473485603934023e-08,
  'z': 6.76258,
  'se': 0.07687,
  'pa': 0.58494,
  'pe': 0.1356},
 'weights': array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]),
 'categories': [0.0,
  0.2,
  0.3333333333333333,
  0.4,
  0.5,
  0.6,
  0.6666666666666666,
  0.8,
  0.8333333333333334,
  1.0]}

In [106]:
display(percentage_agreement(coverage_aggregated_np.T))

(0.8666666666666667, 0.86)